Workflow followed according to: https://gatk.broadinstitute.org/hc/en-us/articles/360035531192-RNAseq-short-variant-discovery-SNPs-Indels

Github workflow for GATK4 here: https://github.com/gatk-workflows/gatk4-rnaseq-germline-snps-indels/blob/master/gatk4-rna-best-practices.wdl

In [1]:
import os
import pysam
import pandas as pd
from tqdm import tqdm
from varseek.utils import convert_chromosome_value_to_int_when_possible, calculate_metrics, draw_confusion_matrix, create_venn_diagram, calculate_sensitivity_specificity, create_stratified_metric_bar_plot, add_vcf_info_to_cosmic_tsv

In [2]:
import random

def generate_random_fastq(output_path, num_sequences=5000, seq_length=150, quality_score="I"):
    if not os.path.exists(output_path):
        if not os.path.exists(os.path.dirname(output_path)):
            os.makedirs(os.path.dirname(output_path))
        bases = ['A', 'C', 'G', 'T']
        with open(output_path, "w") as fastq_file:
            for i in range(num_sequences):
                # Generate a random sequence of the specified length
                sequence = ''.join(random.choices(bases, k=seq_length))
                
                # Create a quality string of the same length, filled with the specified quality score
                quality = quality_score * seq_length
                
                # Write the FASTQ entry
                fastq_file.write(f"@seq_{i + 1}\n")
                fastq_file.write(f"{sequence}\n")
                fastq_file.write("+\n")
                fastq_file.write(f"{quality}\n")

In [3]:
synthetic_read_fastq = "/home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_200mcrs_k59_nov16/synthetic_reads.fq"  #!!! update path
unique_mcrs_df_path = "/home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_200mcrs_k59_nov16/unique_mcrs_df.csv"  #!!! update path
gatk_parent = "/home/jrich/data/varseek_data/gatk_nov17"  #!!! update for each run
# generate_random_fastq(synthetic_read_fastq)

threads = 32
read_length = 150
mutation_source = "cdna"  # "cdna", "cds"

cosmic_tsv = "/home/jrich/data/varseek_data/reference/cosmic/CancerMutationCensus_AllData_Tsv_v100_GRCh37/CancerMutationCensus_AllData_v100_GRCh37.tsv"
cosmic_cdna_info_csv = "/home/jrich/data/varseek_data/reference/cosmic/CancerMutationCensus_AllData_Tsv_v100_GRCh37/CancerMutationCensus_AllData_v100_GRCh37_mutation_workflow_with_cdna.csv"

# if these paths don't exist then they will be created
reference_genome_fasta = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa"
reference_genome_gtf = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf"
genomes1000_vcf = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf"
ensembl_germline_vcf = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/homo_sapiens.vcf"
star_genome_dir = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/star_reference"

STAR = "/home/jrich/opt/STAR-2.7.11b/source/STAR"
java = "/home/jrich/opt/java/jdk-17.0.12+7/bin/java"
picard_jar = "/home/jrich/opt/picard/build/libs/picard.jar"
gatk = "/home/jrich/opt/gatk-4.6.0.0/gatk"

In [4]:
java_home = os.path.dirname(os.path.dirname(java))

os.environ['JAVA_HOME'] = java_home
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:" + os.environ['PATH']

os.makedirs(star_genome_dir, exist_ok=True)

alignment_folder = f"{gatk_parent}/alignment"
os.makedirs(alignment_folder, exist_ok=True)

gatk_supporting_files = f"{gatk_parent}/supporting_files"
os.makedirs(gatk_supporting_files, exist_ok=True)

ensembl_germline_vcf_filtered = ensembl_germline_vcf.replace(".vcf", "_filtered.vcf")

out_file_name_prefix = f"{alignment_folder}/sample_"

vcf_folder = f"{gatk_parent}/vcfs"
haplotypecaller_folder = f"{vcf_folder}/haplotypecaller"
mutect2_folder = f"{vcf_folder}/mutect2"

os.makedirs(vcf_folder, exist_ok=True)
os.makedirs(haplotypecaller_folder, exist_ok=True)
os.makedirs(mutect2_folder, exist_ok=True)

aligned_and_unmapped_bam = f"{out_file_name_prefix}Aligned.sortedByCoord.out.bam"
aligned_only_bam = f"{alignment_folder}/aligned_only.bam"
unmapped_bam = f"{alignment_folder}/unmapped.bam"
merged_bam = f"{alignment_folder}/merged.bam"

marked_duplicates_bam = f"{alignment_folder}/marked_duplicates.bam"
marked_dup_metrics_txt = f"{alignment_folder}/marked_dup_metrics.txt"

split_n_cigar_reads_bam = f"{alignment_folder}/split_n_cigar_reads.bam"
recal_data_table = f"{alignment_folder}/recal_data.table"
recalibrated_bam = f"{alignment_folder}/recalibrated.bam"
covariates_plot = f"{alignment_folder}/AnalyzeCovariates.pdf"
haplotypecaller_unfiltered_vcf = f"{haplotypecaller_folder}/haplotypecaller_output_unfiltered.g.vcf.gz"

haplotypecaller_filtered_vcf = f"{haplotypecaller_folder}/haplotypecaller_output_filtered.vcf.gz"
haplotypecaller_filtered_applied_vcf = f"{haplotypecaller_folder}/haplotypecaller_output_filtered_applied.vcf.gz"

panel_of_normals_vcf = f"{gatk_supporting_files}/1000g_pon.hg38.vcf.gz"
panel_of_normals_vcf_filtered = f"{gatk_supporting_files}/1000g_pon.hg38_filtered.vcf.gz"
mutect2_unfiltered_vcf = f"{mutect2_folder}/mutect2_output_unfiltered.g.vcf.gz"
mutect2_filtered_vcf = f"{mutect2_folder}/mutect2_output_filtered.vcf.gz"

In [5]:
if not os.path.exists(reference_genome_fasta):
    !wget -O {reference_genome_fasta}.gz https://ftp.ensembl.org/pub/grch37/release-93/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz && gunzip {reference_genome_fasta}.gz

if not os.path.exists(reference_genome_gtf):
    !wget -O {reference_genome_gtf}.gz https://ftp.ensembl.org/pub/grch37/release-93/gtf/homo_sapiens/Homo_sapiens.GRCh37.87.gtf.gz && gunzip {reference_genome_gtf}.gz

if not os.path.exists(genomes1000_vcf):
    !wget -O {genomes1000_vcf}.gz https://ftp.ensembl.org/pub/grch37/release-93/variation/vcf/homo_sapiens/1000GENOMES-phase_3.vcf.gz && gunzip {genomes1000_vcf}.gz
    
if not os.path.exists(ensembl_germline_vcf):
    !wget -O {ensembl_germline_vcf}.gz https://ftp.ensembl.org/pub/grch37/release-93/variation/vcf/homo_sapiens/homo_sapiens.vcf.gz && gunzip {ensembl_germline_vcf}.gz

if not os.path.exists(ensembl_germline_vcf_filtered):
    ensembl_germline_vcf_temp1 = ensembl_germline_vcf.replace(".vcf", "_temp1.vcf")
    # ensembl_germline_vcf_temp2 = ensembl_germline_vcf.replace(".vcf", "_temp2.vcf")
    !grep -vP '^[^\t]*\t[^\t]*\t[^\t]*\t[^\t]*[WH]|^[^\t]*\t[^\t]*\t[^\t]*\t[^\t]*\t[^\t]*[WH]' $ensembl_germline_vcf > $ensembl_germline_vcf_temp1
    !grep -vP '^[^\t]*\t[^\t]*\t[^\t]*\t\t|^[^\t]*\t[^\t]*\t[^\t]*\t[^\t]*\t\t' $ensembl_germline_vcf_temp1 > $ensembl_germline_vcf_filtered
    !rm $ensembl_germline_vcf_temp1

# if not os.path.exists(panel_of_normals_vcf):
#     !wget -P {gatk_supporting_files} https://storage.googleapis.com/gatk-best-practices/somatic-hg38/1000g_pon.hg38.vcf.gz
#     !wget -P {gatk_supporting_files} https://storage.googleapis.com/gatk-best-practices/somatic-hg38/1000g_pon.hg38.vcf.gz.tbi

# if not os.path.exists(panel_of_normals_vcf_filtered):
#     decompressed_pon_vcf = panel_of_normals_vcf.replace(".vcf.gz", ".vcf")
#     !gunzip -c $panel_of_normals_vcf > $decompressed_pon_vcf
    
#     # Remove 'chr' prefix from data lines
#     !sed -i 's/^chr//' $decompressed_pon_vcf

#     # Remove 'chr' prefix from contig headers in the VCF
#     !sed -i '/^##contig/s/ID=chr/ID=/' $decompressed_pon_vcf

#     # gunzip again
#     !bgzip -c $decompressed_pon_vcf > $panel_of_normals_vcf_filtered  # TODO: get bgzip installed

1. Mapping to the Reference with STAR

In [6]:
read_length_minus_one = read_length - 1

if not os.listdir(star_genome_dir):
    !$STAR \
        --runThreadN $threads \
        --runMode genomeGenerate \
        --genomeDir $star_genome_dir \
        --genomeFastaFiles $reference_genome_fasta \
        --sjdbGTFfile $reference_genome_gtf \
        --sjdbOverhang $read_length_minus_one

In [7]:
#* --outSAMmapqUnique 60 - change from default of 255 to avoid colliding with some aligners' use of 255 as a special value

!$STAR \
    --runThreadN $threads \
    --genomeDir $star_genome_dir \
    --readFilesIn $synthetic_read_fastq \
    --sjdbOverhang $read_length_minus_one \
    --outFileNamePrefix $out_file_name_prefix \
    --outSAMtype BAM SortedByCoordinate \
    --outSAMunmapped Within \
    --outSAMmapqUnique 60 \
    --twopassMode Basic



	/home/jrich/opt/STAR-2.7.11b/source/STAR --runThreadN 32 --genomeDir /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/star_reference --readFilesIn /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_200mcrs_k59_nov16/synthetic_reads.fq --sjdbOverhang 149 --outFileNamePrefix /home/jrich/data/varseek_data/gatk_nov17/alignment/sample_ --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outSAMmapqUnique 60 --twopassMode Basic
	STAR version: 2.7.11b   compiled: 2024-07-22T09:22:47-0700 dator:/home/jrich/opt/STAR-2.7.11b/source
Nov 20 16:11:16 ..... started STAR run
Nov 20 16:11:16 ..... loading genome
Nov 20 16:11:57 ..... started 1st pass mapping
Nov 20 16:12:00 ..... finished 1st pass mapping
Nov 20 16:12:01 ..... inserting junctions into the genome indices
Nov 20 16:13:32 ..... started mapping
Nov 20 16:13:36 ..... finished mapping
Nov 20 16:13:37 ..... started sorting BAM
Nov 20 16:13:38 ..... finished successfully


Separate unmapped reads into its own BAM file

In [8]:
# with pysam.AlignmentFile(aligned_and_unmapped_bam, "rb") as bam_in: 
#     # unmapped_header = bam_in.header.to_dict()
#     # if 'PG' in unmapped_header:
#     #     del unmapped_header['PG']
           
#     with pysam.AlignmentFile(aligned_only_bam, "wb", template=bam_in) as bam_aligned_out, pysam.AlignmentFile(unmapped_bam, "wb", template=bam_in) as bam_unmapped_out:
#         for read in bam_in:
#             if read.is_unmapped:
#                 bam_unmapped_out.write(read)  # Write unmapped read to the unmapped BAM file
#             else:
#                 bam_aligned_out.write(read)  # Write aligned read to the aligned BAM file

# print(f"Unmapped reads written to {unmapped_bam}")

#

In [9]:
!$java -jar $picard_jar FastqToSam \
    -FASTQ $synthetic_read_fastq \
    -OUTPUT $unmapped_bam \
    -READ_GROUP_NAME rg1 \
    -SAMPLE_NAME sample1 \
    -LIBRARY_NAME lib1 \
    -PLATFORM_UNIT unit1 \
    -PLATFORM ILLUMINA \
    -SEQUENCING_CENTER center1

16:13:40.706 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Nov 20 16:13:40 PST 2024] FastqToSam --FASTQ /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_200mcrs_k59_nov16/synthetic_reads.fq --OUTPUT /home/jrich/data/varseek_data/gatk_nov17/alignment/unmapped.bam --READ_GROUP_NAME rg1 --SAMPLE_NAME sample1 --LIBRARY_NAME lib1 --PLATFORM_UNIT unit1 --PLATFORM ILLUMINA --SEQUENCING_CENTER center1 --USE_SEQUENTIAL_FASTQS false --SORT_ORDER queryname --MIN_Q 0 --MAX_Q 93 --STRIP_UNPAIRED_MATE_NUMBER false --ALLOW_AND_IGNORE_EMPTY_LINES false --ALLOW_EMPTY_FASTQ false --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 5 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false
[Wed Nov 20 16:13:40 PST 2024] Executing as jrich@da

2. MergeBamAlignment

In [10]:
reference_genome_dict = reference_genome_fasta.replace(".fa", ".dict")

!$java -jar $picard_jar CreateSequenceDictionary \
    -R $reference_genome_fasta \
    -O $reference_genome_dict

16:13:42.612 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Nov 20 16:13:42 PST 2024] CreateSequenceDictionary --OUTPUT /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.dict --REFERENCE /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUENCES 2147483647 --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 5 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false
[Wed Nov 20 16:13:42 PST 2024] Executing as jrich@dator on Linux 3.10.0-1127.13.1.el7.x86_64 amd64; OpenJDK 64-Bit Server VM 17.0.12+7; Deflater: Intel; Inflater: Intel; Provider GCS is available; Picard version

In [11]:
!$java -jar $picard_jar MergeBamAlignment \
    --ALIGNED_BAM $aligned_and_unmapped_bam \
    --UNMAPPED_BAM $unmapped_bam \
    --OUTPUT $merged_bam \
    --REFERENCE_SEQUENCE $reference_genome_fasta \
    --SORT_ORDER coordinate \
    --INCLUDE_SECONDARY_ALIGNMENTS false \
    --VALIDATION_STRINGENCY SILENT



16:13:43.627 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Nov 20 16:13:43 PST 2024] MergeBamAlignment --UNMAPPED_BAM /home/jrich/data/varseek_data/gatk_nov17/alignment/unmapped.bam --ALIGNED_BAM /home/jrich/data/varseek_data/gatk_nov17/alignment/sample_Aligned.sortedByCoord.out.bam --OUTPUT /home/jrich/data/varseek_data/gatk_nov17/alignment/merged.bam --SORT_ORDER coordinate --INCLUDE_SECONDARY_ALIGNMENTS false --VALIDATION_STRINGENCY SILENT --REFERENCE_SEQUENCE /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --ADD_PG_TAG_TO_READS true --PAIRED_RUN true --CLIP_ADAPTERS true --IS_BISULFITE_SEQUENCE false --ALIGNED_READS_ONLY false --MAX_INSERTIONS_OR_DELETIONS 1 --ATTRIBUTES_TO_REVERSE OQ --ATTRIBUTES_TO_REVERSE U2 --ATTRIBUTES_TO_REVERSE_COMPLEMENT E2 --ATTRIBUTES_TO_REVERSE_COMPLEMENT SQ --READ1_TRIM 0 --R

3. MarkDuplicates

In [12]:
!$java -jar $picard_jar MarkDuplicates \
      --INPUT $merged_bam \
      --OUTPUT $marked_duplicates_bam \
      --METRICS_FILE $marked_dup_metrics_txt \
      --CREATE_INDEX true \
      --VALIDATION_STRINGENCY SILENT

16:13:53.536 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Nov 20 16:13:53 PST 2024] MarkDuplicates --INPUT /home/jrich/data/varseek_data/gatk_nov17/alignment/merged.bam --OUTPUT /home/jrich/data/varseek_data/gatk_nov17/alignment/marked_duplicates.bam --METRICS_FILE /home/jrich/data/varseek_data/gatk_nov17/alignment/marked_dup_metrics.txt --VALIDATION_STRINGENCY SILENT --CREATE_INDEX true --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --DUPLEX_UMI false --FLOW_MODE false --FLOW_DUP_STRATEGY FLOW_QUALITY_SUM_STRATEGY --USE_END_IN_UNPAIRED_READS false --USE_UNPAIRED_CLIPPED_END false --UNPAIRED_END_UNCERTAINTY 0 --UNPAIRED_START_UNCERTAINTY 0 --FLOW_SKIP_FIRST_N_FLOWS 0 --FLOW_Q_IS_KN

4. SplitNCigarReads

In [13]:
_ = pysam.faidx(reference_genome_fasta)

!$gatk SplitNCigarReads \
    -R $reference_genome_fasta \
    -I $marked_duplicates_bam \
    -O $split_n_cigar_reads_bam

Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar SplitNCigarReads -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk_nov17/alignment/marked_duplicates.bam -O /home/jrich/data/varseek_data/gatk_nov17/alignment/split_n_cigar_reads.bam
16:14:12.055 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
16:14:12.299 INFO  SplitNCigarReads - ------------------------------------------------------------
16:14:12.304 INFO  SplitNCigarReads - The Genome Analysis Toolkit (GATK) v4.6.0.0
16:14:12.304 INFO  SplitNCigarRead

In [14]:
import pandas as pd
import numpy as np

# DataFrame with 1000 tuples of length 1
df_tuples = pd.DataFrame({"col": [(i,i+1,i+2) for i in range(1000)]})

# DataFrame with 1000 NumPy arrays of length 1
df_arrays = pd.DataFrame({"col": [np.array([i,i+1,i+2]) for i in range(1000)]})

df_sets = pd.DataFrame({"col": [set([i,i+1,i+2]) for i in range(1000)]})

df_lists = pd.DataFrame({"col": [[i,i+1,i+2] for i in range(1000)]})

print("Memory usage of DataFrame with tuples:")
print(df_tuples.memory_usage(deep=True))

print("\nMemory usage of DataFrame with NumPy arrays:")
print(df_arrays.memory_usage(deep=True))

print("\nMemory usage of DataFrame with sets:")
print(df_sets.memory_usage(deep=True))

print("\nMemory usage of DataFrame with lists:")
print(df_lists.memory_usage(deep=True))

Memory usage of DataFrame with tuples:
Index      128
col      56000
dtype: int64

Memory usage of DataFrame with NumPy arrays:
Index       128
col      144000
dtype: int64

Memory usage of DataFrame with sets:
Index       128
col      208000
dtype: int64

Memory usage of DataFrame with lists:
Index      128
col      72000
dtype: int64


In [15]:
bus_df

NameError: name 'bus_df' is not defined

5. BaseRecalibrator

In [ ]:
##  -I in old version, -F in new version
# if not os.path.exists(f"{ensembl_germline_vcf_filtered}.idx"):
#     !$gatk IndexFeatureFile -I $ensembl_germline_vcf_filtered  # TODO: ~81092000/324780532 processed before error - current error being a duplicate G in REF/ALT column

if not os.path.exists(f"{genomes1000_vcf}.idx"):
    !$gatk IndexFeatureFile -I $genomes1000_vcf

# TODO: uncomment once I install bgzip
# if not os.path.exists(f"{panel_of_normals_vcf_filtered}.idx"):
#     !gatk IndexFeatureFile -I $panel_of_normals_vcf_filtered

In [ ]:
# TODO: when I fix !$gatk IndexFeatureFile -I $ensembl_germline_vcf_filtered, add back --known-sites $ensembl_germline_vcf_filtered
!$gatk BaseRecalibrator \
    -I $split_n_cigar_reads_bam \
    -R $reference_genome_fasta \
    --use-original-qualities \
    --known-sites $genomes1000_vcf \
    -O $recal_data_table

# -known-sites ${dbSNP_vcf} \
# -known-sites ${sep=" --known-sites " known_indels_sites_VCFs}

Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar BaseRecalibrator -I /home/jrich/data/varseek_data/gatk_nov16/alignment/split_n_cigar_reads.bam -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --use-original-qualities --known-sites /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf -O /home/jrich/data/varseek_data/gatk_nov16/alignment/recal_data.table
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java R

6. Apply Recalibration

In [ ]:
!$gatk ApplyBQSR \
    --add-output-sam-program-record \
    -R $reference_genome_fasta \
    -I $split_n_cigar_reads_bam \
    --use-original-qualities \
    --bqsr-recal-file $recal_data_table \
    -O $recalibrated_bam



Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar ApplyBQSR --add-output-sam-program-record -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk_nov16/alignment/split_n_cigar_reads.bam --use-original-qualities --bqsr-recal-file /home/jrich/data/varseek_data/gatk_nov16/alignment/recal_data.table -O /home/jrich/data/varseek_data/gatk_nov16/alignment/recalibrated.bam
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of th

7. AnalyzeCovariates

In [ ]:
!$gatk AnalyzeCovariates \
    -bqsr $recal_data_table \
    -plots $covariates_plot



Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar AnalyzeCovariates -bqsr /home/jrich/data/varseek_data/gatk_nov16/alignment/recal_data.table -plots /home/jrich/data/varseek_data/gatk_nov16/alignment/AnalyzeCovariates.pdf
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0


8a. HaplotypeCaller

In [ ]:

!$gatk --java-options "-Xmx4g" HaplotypeCaller  \
    -R $reference_genome_fasta \
    -I $recalibrated_bam \
    -O $haplotypecaller_unfiltered_vcf \
    --standard-min-confidence-threshold-for-calling 20 \
    -dont-use-soft-clipped-bases \



Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -Xmx4g -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar HaplotypeCaller -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk_nov16/alignment/recalibrated.bam -O /home/jrich/data/varseek_data/gatk_nov16/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz --standard-min-confidence-threshold-for-calling 20 -dont-use-soft-clipped-bases
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only 

8.5a. MergeVcfs

In [ ]:
# merged_vcf = haplotypecaller_merged.vcf
# 
# !$java -jar $picard_jar MergeVcfs \
#     --INPUT $vcf1 \
#     --INPUT $vcf2 \
#     --OUTPUT $merged_vcf

9a. VariantFiltration

In [ ]:
# cosmic_vcf = ""

!$gatk VariantFiltration \
    -R $reference_genome_fasta \
    -V $haplotypecaller_unfiltered_vcf \
    -O $haplotypecaller_filtered_vcf \
    --window 35 \
    --cluster 3 \
    --filter-name "FS" \
    --filter "FS > 30.0" \
    --filter-name "QD" \
    --filter "QD < 2.0"

    # --mask $cosmic_vcf \
    # --mask-name "COSMIC" \
    # --filter-not-in-mask \



Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar VariantFiltration -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -V /home/jrich/data/varseek_data/gatk_nov16/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz -O /home/jrich/data/varseek_data/gatk_nov16/vcfs/haplotypecaller/haplotypecaller_output_filtered.vcf.gz --window 35 --cluster 3 --filter-name FS --filter FS > 30.0 --filter-name QD --filter QD < 2.0
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61

10a. Do the filtering

In [ ]:
!$gatk SelectVariants \
     -V $haplotypecaller_filtered_vcf \
     --exclude-filtered true \
     -O $haplotypecaller_filtered_applied_vcf

Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar SelectVariants -V /home/jrich/data/varseek_data/gatk_nov16/vcfs/haplotypecaller/haplotypecaller_output_filtered.vcf.gz --exclude-filtered true -O /home/jrich/data/varseek_data/gatk_nov16/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0


8b. Mutect2

In [ ]:
# consider adding --disable-read-filter
# TODO: add --panel-of-normals $panel_of_normals_vcf_filtered once I install bgzip
!$gatk Mutect2 \
    -R $reference_genome_fasta \
    -I $recalibrated_bam \
    -O $mutect2_unfiltered_vcf \
    --min-base-quality-score 20



Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar Mutect2 -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk_nov16/alignment/recalibrated.bam -O /home/jrich/data/varseek_data/gatk_nov16/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz --min-base-quality-score 20
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0


9b. FilterMutectCalls

In [ ]:
# # if multiple stats files:
# !$gatk MergeMutectStats -stats unfiltered1.vcf.stats -stats unfiltered2.vcf.stats -O unfiltered.vcf.stats

# stats_file = f"{mutect2_unfiltered_vcf}.stats"
# --stats $stats_file
!$gatk FilterMutectCalls \
    -R $reference_genome_fasta \
    -V $mutect2_unfiltered_vcf \
    -O $mutect2_filtered_vcf



Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar FilterMutectCalls -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -V /home/jrich/data/varseek_data/gatk_nov16/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz -O /home/jrich/data/varseek_data/gatk_nov16/vcfs/mutect2/mutect2_output_filtered.vcf.gz
Error: LinkageError occurred while loading main class org.broadinstitute.hellbender.Main
	java.lang.UnsupportedClassVersionError: org/broadinstitute/hellbender/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0


In [ ]:
# def create_sample_vcf(output_path):
#     with open(output_path, "w") as vcf_file:
#         # Write VCF headers
#         vcf_file.write("##fileformat=VCFv4.2\n")
#         vcf_file.write("##source=SampleVCFGenerator\n")
#         vcf_file.write("##reference=GRCh37\n")
#         vcf_file.write("##INFO=<ID=DP,Number=1,Type=Integer,Description=\"Total Depth\">\n")
#         vcf_file.write("##INFO=<ID=AF,Number=A,Type=Float,Description=\"Allele Frequency\">\n")
#         vcf_file.write("##FILTER=<ID=PASS,Description=\"All filters passed\">\n")
#         vcf_file.write("##FORMAT=<ID=GT,Number=1,Type=String,Description=\"Genotype\">\n")
#         vcf_file.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tsample1\n")
        
#         # Write sample variant entries
#         variants = [
#             ("1", 123456, ".", "G", "A", 50, "PASS", "DP=100;AF=0.5", "GT", "0/1"),
#             ("1", 234567, ".", "C", "T", 60, "PASS", "DP=200;AF=0.3", "GT", "1/1"),
#             ("2", 345678, ".", "T", "G", 70, "PASS", "DP=150;AF=0.1", "GT", "0/1"),
#             ("X", 456789, ".", "A", "C", 80, "PASS", "DP=120;AF=0.05", "GT", "0/0")
#         ]
        
#         for chrom, pos, var_id, ref, alt, qual, fltr, info, fmt, sample in variants:
#             vcf_file.write(f"{chrom}\t{pos}\t{var_id}\t{ref}\t{alt}\t{qual}\t{fltr}\t{info}\t{fmt}\t{sample}\n")

# # Usage
# create_sample_vcf("/home/jrich/data/varseek_data/gatk/sample.vcf")

# df_sample = vcf_to_dataframe("/home/jrich/data/varseek_data/gatk/sample.vcf")

# JUMP TO HERE

In [ ]:
from varseek.utils import vcf_to_dataframe

# Convert VCF to DataFrame
df_hap = vcf_to_dataframe(haplotypecaller_filtered_applied_vcf, additional_columns = False)
df_mut = vcf_to_dataframe(mutect2_filtered_vcf, additional_columns = False)



[E::hts_open_format] Failed to open file "/home/jrich/data/varseek_data/gatk_nov16/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz" : No such file or directory


FileNotFoundError: [Errno 2] could not open variant file `b'/home/jrich/data/varseek_data/gatk_nov16/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz'`: No such file or directory

In [ ]:
cosmic_df = add_vcf_info_to_cosmic_tsv(cosmic_tsv=cosmic_tsv, reference_genome_fasta=reference_genome_fasta, cosmic_df_out = None, cosmic_cdna_info_csv = cosmic_cdna_info_csv, mutation_source = "cdna")

In [ ]:
# load in unique_mcrs_df
unique_mcrs_df = pd.read_csv(unique_mcrs_df_path)
unique_mcrs_df.rename(columns={'TP': 'TP_vk', 'FP': 'FP_vk', 'TN': 'TN_vk', 'FN': 'FN_vk'}, inplace=True)

/tmp/ipykernel_188830/1989552023.py:2: DtypeWarning: Columns (21,36,38,49,50,54,56) have mixed types. Specify dtype option on import or set low_memory=False.
  unique_mcrs_df = pd.read_csv(unique_mcrs_df_path)


In [ ]:
# filtered_df = cosmic_df[(cosmic_df['POS'].astype(int) >= 3699230) & (cosmic_df['POS'].astype(int) <= 3699239)]
# filtered_df

In [ ]:
cosmic_tp = unique_mcrs_df.loc[unique_mcrs_df['TP_vk']]

In [ ]:
#  take the intersection of COSMIC and STAR dfs based on CHROM, POS, REF, ALT - but keep the ID from the COSMIC vcf
mut_cosmic_merged_df = pd.merge(df_mut, cosmic_df, 
                     on=['CHROM', 'POS', 'REF', 'ALT'], 
                     how='inner',
                     suffixes=('_df1', '_df2'))

mut_cosmic_merged_df = mut_cosmic_merged_df.drop(columns=['ID_df1']).rename(columns={'ID_df2': 'ID'})

id_set_mut = set(mut_cosmic_merged_df['ID'])



hap_cosmic_merged_df = pd.merge(df_hap, cosmic_df, 
                     on=['CHROM', 'POS', 'REF', 'ALT'], 
                     how='inner',
                     suffixes=('_df1', '_df2'))

hap_cosmic_merged_df = hap_cosmic_merged_df.drop(columns=['ID_df1']).rename(columns={'ID_df2': 'ID'})

id_set_hap = set(hap_cosmic_merged_df['ID'])

KeyError: 'CHROM'

In [ ]:
number_of_mutations_mutect = len(df_mut)
number_of_cosmic_mutations_mutect = len(mut_cosmic_merged_df)
number_of_mutations_haplotypecaller = len(df_hap)
number_of_cosmic_mutations_haplotypecaller = len(hap_cosmic_merged_df)

In [ ]:
unique_mcrs_df['mutation_detected_gatk_mutect2'] = unique_mcrs_df['reference_header'].isin(id_set_mut)  #!!! ensure 'reference_header' is correct here  # keep in mind that my IDs are the mutation headers (ENST...), NOT mcrs headers or mcrs ids

unique_mcrs_df['TP'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_mutect2'])
unique_mcrs_df['FP'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_mutect2'])
unique_mcrs_df['FN'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_mutect2'])
unique_mcrs_df['TN'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_mutect2'])

mutect_stat_path = f"{gatk_parent}/reference_metrics_mutect2.txt"
metric_dictionary_reference = calculate_metrics(unique_mcrs_df, header_name = "mcrs_header", check_assertions = False, out = mutect_stat_path)
draw_confusion_matrix(metric_dictionary_reference)

true_set = set(unique_mcrs_df.loc[unique_mcrs_df['included_in_synthetic_reads_mutant'], 'mcrs_header'])
positive_set = set(unique_mcrs_df.loc[unique_mcrs_df['mutation_detected_gatk_mutect2'], 'mcrs_header'])
create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_cosmic_only_mutect2.png")


noncosmic_mutation_id_set = {f'mutect_fp_{i}' for i in range(1, number_of_mutations_mutect - number_of_cosmic_mutations_mutect + 1)}
positive_set_including_noncosmic_mutations = positive_set.union(noncosmic_mutation_id_set)

FP_including_noncosmic = metric_dictionary_reference['FP'] + len(positive_set_including_noncosmic_mutations)
accuracy, sensitivity, specificity = calculate_sensitivity_specificity(metric_dictionary_reference['TP'], metric_dictionary_reference['TP'], FP_including_noncosmic, metric_dictionary_reference['TP'])

with open(mutect_stat_path, "a") as file:
    file.write(f"FP including non-cosmic: {FP_including_noncosmic}\n")
    file.write(f"accuracy including non-cosmic: {accuracy}\n")
    file.write(f"specificity including non-cosmic: {specificity}\n")

create_venn_diagram(true_set, positive_set_including_noncosmic_mutations, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_including_noncosmics_mutect2.png")

create_stratified_metric_bar_plot(unique_mcrs_df, 'number_of_reads_mutant', 'accuracy', overall_metric = metric_dictionary_reference['accuracy'], log_x_axis = False, out_path = f"{plot_output_folder}/accuracy_vs_number_of_reads_mutant.png")
#!!! create similar plots for y in {sensitivity, specificity}, and x in {number_of_reads_wt, tumor_purity} and determine cutoffs for which GATK is reliable

unique_mcrs_df.rename(columns={'TP': 'TP_gatk_mutect2', 'FP': 'FP_gatk_mutect2', 'TN': 'TN_gatk_mutect2', 'FN': 'FN_gatk_mutect2'}, inplace=True)

In [ ]:
unique_mcrs_df['mutation_detected_gatk_haplotypecaller'] = unique_mcrs_df['reference_header'].isin(id_set_hap)

unique_mcrs_df['TP'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])
unique_mcrs_df['FP'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])
unique_mcrs_df['FN'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])
unique_mcrs_df['TN'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])

haplotypecaller_stat_path = f"{gatk_parent}/reference_metrics_haplotypecaller.txt"
metric_dictionary_reference = calculate_metrics(unique_mcrs_df, header_name = "mcrs_header", check_assertions = False, out = haplotypecaller_stat_path)
draw_confusion_matrix(metric_dictionary_reference)

true_set = set(unique_mcrs_df.loc[unique_mcrs_df['included_in_synthetic_reads_mutant'], 'mcrs_header'])
positive_set = set(unique_mcrs_df.loc[unique_mcrs_df['mutation_detected_gatk_haplotypecaller'], 'mcrs_header'])
create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_cosmic_only_haplotypecaller.png")



noncosmic_mutation_id_set = {f'haplotypecaller_fp_{i}' for i in range(1, number_of_mutations_haplotypecaller - number_of_cosmic_mutations_haplotypecaller + 1)}
positive_set_including_noncosmic_mutations = positive_set.union(noncosmic_mutation_id_set)

FP_including_noncosmic = metric_dictionary_reference['FP'] + len(positive_set_including_noncosmic_mutations)
accuracy, sensitivity, specificity = calculate_sensitivity_specificity(metric_dictionary_reference['TP'], metric_dictionary_reference['TP'], FP_including_noncosmic, metric_dictionary_reference['TP'])

with open(haplotypecaller_stat_path, "a") as file:
    file.write(f"FP including non-cosmic: {FP_including_noncosmic}\n")
    file.write(f"accuracy including non-cosmic: {accuracy}\n")
    file.write(f"specificity including non-cosmic: {specificity}\n")

create_venn_diagram(true_set, positive_set_including_noncosmic_mutations, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_including_noncosmics_haplotypecaller.png")

create_stratified_metric_bar_plot(unique_mcrs_df, 'number_of_reads_mutant', 'accuracy', overall_metric = metric_dictionary_reference['accuracy'], log_x_axis = False, out_path = f"{plot_output_folder}/accuracy_vs_number_of_reads_mutant.png")
#!!! create similar plots for y in {sensitivity, specificity}, and x in {number_of_reads_wt, tumor_purity} and determine cutoffs for which GATK is reliable

unique_mcrs_df.rename(columns={'TP': 'TP_gatk_haplotypecaller', 'FP': 'FP_gatk_haplotypecaller', 'TN': 'TN_gatk_haplotypecaller', 'FN': 'FN_gatk_haplotypecaller'}, inplace=True)